In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading dataset 
df = pd.read_csv('/kaggle/input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv')
df.info()

In [ ]:
# Getting first 5 rows 
df.head()

In [ ]:
# Checking no of states & UT .
print('There are {} no of states & union territories info . '.format(len(df['State/UTs'])))
print('And they are: \n {}'.format(df['State/UTs'].unique()))

In [ ]:
# Checking which states or UT has the highest active cases .
plt.figure(figsize = (10,9))
plt.plot(df['Active'],df['State/UTs'])
plt.xlabel('No of Active Cases')
plt.ylabel('States or Union Territories')

<h3>So,we can clearly figure out Ladakh,Kerala,Karnataka has the highest active cases.</h3>

In [ ]:
# Checking which states or UT has the highest discharged cases .
plt.figure(figsize = (10,9))
plt.plot(df['Discharged'],df['State/UTs'])
plt.xlabel('Discharged Cases')
plt.ylabel('States or Union Territories')

<h3>Manipur,Maharashtra,Madhya Pradesh has the highest discharge numbers,
followed by Ladakh , Kerala , Karnataka and etc </h3> 

In [ ]:
# Checking which states or UT has the highest death cases .
plt.figure(figsize = (10,9))
plt.plot(df['Deaths'],df['State/UTs'])
plt.xlabel('Deaths Cases')
plt.ylabel('States or Union Territories')

<h3>We cannot analyse Deaths by No of discharge cases because we can see No of discharge cases in Maharashtra was the most but still Maharashtra has the highest death cases <.h3>

In [ ]:
ratio_df = df.iloc[:,5:]
ratio_df.head()

In [ ]:
plt.figure(figsize = (10,9))
plt.plot(ratio_df,df['State/UTs'] , label = ('Active','Discharge','Death'))
plt.xlabel("Ratio's of Active,Discharge and Death cases.")
plt.ylabel('States or Union Territories')
plt.legend()
plt.show()

<h3>As we can see death ratios are very minimal campared to active ratio's</h3>

<h3> So let's try to predict No of deaths </h3>

In [ ]:
#We have got str values in States/UTs columns
s = (df.dtypes == 'object')
object_cols = s[s].index
print("Categorical Variable :{} ".format(object_cols))

In [ ]:
#so lets convert them into int values.
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
df[object_cols] = oe.fit_transform(df[object_cols])

In [ ]:
# So lets check the values which are assigned by Ordinal Encoder to object column.

df[object_cols]

In [ ]:
# Let's one again check our dataframe .
df.sample(2)

In [ ]:
# Divide features and labels for train_test_split
df_features = df.iloc[:,:]
df_label = df['Deaths']

In [ ]:
#Since We have label as Deaths so we need to drop it from the features.
df_features = df_features.drop('Deaths',axis =1 )
df_features.sample(1)

In [ ]:
# We don't have our features in same scale i.e some values are close to 100 and some are close to 1
# So in order to get the best model we should scale our features.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_features_cols = df_features.columns 
# In scaling , standard scaler will remove the columns from features.
# So we need to add those columns back to the scaled features
df_features[df_features_cols] = sc.fit_transform(df_features[df_features_cols])

In [ ]:
# Lets check our scaled features 
df_features.sample(2)

In [ ]:
# We need to split our dataframe into train_test . 

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df_features,df_label,test_size = 0.2 )
print('Shape of x_train: ',x_train.shape)
print('Shape of y_train:',y_train.shape)
print('Shape of x_test:',x_test.shape)
print('Shape of y_test:',y_test.shape)

In [ ]:
# Let's build ML model out of this data now. 

from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(x_train,y_train)
lr_model.score(x_test,y_test)

In [ ]:
# We need to do predictions on x_test 
y_pred = lr_model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
print("Mean Squared Error: {}".format(mean_squared_error(y_test,y_pred)))
print("R2 Score: {}".format(r2_score(y_test,y_pred)))

<h3> The less the value of Mean Squared Error the better the model is , Whereas for R2 score the closer the value gets to 100 % the better the model.</h3>